# 네이버뉴스

# 메인페이지 헤드라인 기사

In [50]:
def make_headlinedf():
    import requests
    from bs4 import BeautifulSoup as bs
    import re
    import pandas as pd

    while True:
        url = 'https://news.naver.com/'
        headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
        req = requests.get(url, headers = headers)
        target = bs(req.content, 'html.parser')
        main_news = target.find(id='today_main_news') # 헤드라인 뉴스
        news_list = main_news.select('ul.hdline_article_list > li')

        links = []

        for news in news_list:
            links.append(url + news.select_one('div.hdline_cluster_more > a')['href']) # link

        final_link = None
        num = 0
        final_df = pd.DataFrame()

        for link in links[:1]:
            req = requests.get(link, headers = headers)
            target = bs(req.content, 'html.parser')
            num_articles = target.find(class_='cluster_banner_count_icon_num').text
            news_cluster = target.select('div#main_content > div:nth-child(2).cluster > ul')

            보수 = []
            진보 = []
            time_li = []
            media_li = []
            title_li = []
            document_li = []

            for news_list in news_cluster:
                news_li = news_list.select('li')
                for news in news_li:
                    try:
                        news_link = news.select_one('li > dl > dt:nth-child(2) > a')['href']
                    except:
                        news_link = news.select_one('li > dl > dt > a')['href'] # 사진 없는 경우
                    media = news.select_one('span.writing').text # media

                    if media in ['조선일보','동아일보','중앙일보']: # 보수
                        req = requests.get(news_link, headers = headers)
                        target = bs(req.content, 'html.parser')

                        time = target.select_one('span.t11').text
                        title = target.title.text.split(':')[0]
                        document = target.find(id='articleBodyContents').text
                        media_li.append(media)
                        time_li.append(time)
                        title_li.append(title)
                        document_li.append(document)                
                        if media not in 보수:
                            보수.append(media)

                    elif media in ['경향신문','한겨레','한국일보']: # 진보
                        req = requests.get(news_link, headers = headers)
                        target = bs(req.content, 'html.parser')

                        time = target.select_one('span.t11').text
                        title = target.title.text.split(':')[0]
                        document = target.find(id='articleBodyContents').text
                        media_li.append(media)
                        time_li.append(time)
                        title_li.append(title)
                        document_li.append(document)
                        if media not in 진보:
                            진보.append(media)

            # 보수, 진보 언론사 각각 1개 이상, 관련기사 수 가장 많은 기사로 final 저장
            if (len(보수)>=1)&(len(진보)>=1)&(int(num_articles)>num):
                num = int(num_articles)
                final_link = link
                final_df = pd.DataFrame({'time':time_li,'media':media_li,'title':title_li,'document':document_li})

        if len(final_df) == 0:
            continue
        else:
            def to_datetime(time):
                from datetime import datetime
                time = time.replace('오전','AM').replace('오후','PM')
                time = datetime.strptime(time, '%Y.%m.%d. %p %I:%M')
                return time

            final_df['time_list'] = final_df['time'].apply(to_datetime)
            final_df.sort_values(by='time_list',ascending=False,inplace=True) # 최신순으로 정렬
            final_df.drop_duplicates('media',inplace=True) # 동일 언론사 있을 경우, 오래된 기사는 삭제
            final_df.drop('time_list',axis=1,inplace=True)

            if len(final_df[final_df.media.str.contains('조선일보|동아일보')])== 2:
                final_df = final_df.drop(final_df[final_df.media.str.contains('중앙일보')].index) # 중앙일보 삭제
            if len(final_df[final_df.media.str.contains('경향신문|한겨레')])== 2:
                final_df = final_df.drop(final_df[final_df.media.str.contains('한국일보')].index) # 한국일보 삭제
            break

    return final_df

make_headlinedf()

,time,media,title,document
0,2021.11.23. 오후 2:52,중앙일보,"일상회복 대출, 희망대출, 관광융자…업계는 “빚에 빚만 더 얹어”",\n\n\n\n\n 홍남기 경제부총리 겸 기획재정부 장관이 23일 오전 서울 종로...
4,2021.11.23. 오전 8:40,조선일보,"“초과세수 중 5조3000억 소상공인 지원… 1% 저리 대출, 전기료 감면”",\n\n\n\n\n\t\n\t\t\t\t\t\t\t\t홍남기 경제부총리 겸 기획재정...
2,2021.11.23. 오전 8:21,한겨레,홍남기 “초과세수 19조 가운데 5.3조 소상공인 지원에 활용”,\n\n\n\n\n기존 예산까지 더해 12.7조 소상공인·취약계층 지원홍남기 부총리...
1,2021.11.23. 오전 7:50,경향신문,"홍남기 ""12조7000억원 민생대책…손실보상 제외업종 금리 1% 대출""",\n\n\n\n\n\t\n\t[경향신문] 홍남기 부총리 겸 기획재정부 장관이 23일...
